In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

from dataloader.datasets import DataGenerator
from dataloader.utils import get_count_classes, plot_classes, plot_from_one_class
from models.feature_model import load_and_compile_model

SEED = 0

# Load datasets

In [ ]:
data_generator = DataGenerator(
    directory="tiny-imagenet-200/train/",
    batch_size=256,
    image_size=(64, 64),
    shuffle=True,
    seed=SEED,
    validation_split=0.2
)

ds_train, ds_val = data_generator.train, data_generator.val

In [ ]:
x = next(iter(ds_train))

x[0].shape

# Load model

In [ ]:
model = load_and_compile_model(model="efficientnet")
# model = load_and_compile_model(model="resnet50")

In [ ]:
config = {
    "epochs": 10,
    "early_stopping_patience": 3
}

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        patience=config["early_stopping_patience"],
        monitor="val_loss",
        mode="min",
        restore_best_weights=True)
]


# Train the network
history = model.fit(ds_train,
                    epochs=config["epochs"],
                    validation_data=ds_val,
                    callbacks=callbacks)

# Save model

In [ ]:
model.save("saved_models/feature_model")

# Generate embeddings for Tensorboard Projector

In [ ]:
# Evaluate the network
results = model.predict(ds_val)

In [ ]:
# Save test embeddings for visualization in projector
np.savetxt("vecs.tsv", results, delimiter='\t')

with open('meta.tsv', 'w', encoding='utf-8') as f:
    for img, labels in tfds.as_numpy(ds_val):
        [f.write(str(x) + "\n") for x in labels]

# Evaluate model

In [ ]:
model.predict(ds_val)

In [ ]:
model.evaluate(ds_val)

In [ ]:
ds_one_class = ds_val.unbatch().filter(lambda x, y: y==100)

In [ ]:
model.evaluate(ds_one_class.batch(32))